<a href="https://colab.research.google.com/github/framante/NAPDE-project-NN/blob/main/conv_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from keras.layers.core import Reshape

REMEMBER THAT IN PYTHON THE FIRST DIMENSION OF A TENSOR IS THE WIDTH, WHICH I USUALLY CONSIDER AS THE THIRD ONE, while the remaining others are the dimension of a classical matrix n rows x n cols

In [ ]:
#training data

# FINAL STEP: here we implement the version for all FEM nodes (hopefully)

# number of FEM nodes (control nodes in Higa setting)
n_FEMnodes = 51
# number of parameters(mu, beta1, beta2, sigma, f)
n_params = 5
# number of PDEs problems considered
n_PDEs = 500
# number of quadrature nodes for each FEM node
data_size = 32

#import expected output dataset
expected_output = np.loadtxt('SolutionOutput.txt')
print(expected_output.shape) # n FEM nodes x n PDEs x n QUAD nodes

# import input dataset (think about importing n PDEs files instead SolutionInput_1.txt ... SolutionInput_nPDEs.txt)
input = np.loadtxt('SolutionInput.txt')
print(input.shape) # n FEM nodes x n PDEs x n params x n QUAD nodes

(25500, 32)
(127500, 32)


Reshaping the expected output such that it's a tensor where each matrix is (n FEM nodes x n QUAD nodes) and the third dimension is the n PDEs

In [ ]:
expected_output_rshpd = expected_output.reshape([n_PDEs, n_FEMnodes, data_size])
print(expected_output_rshpd.shape) # n PDEs x n FEM nodes x n QUAD nodes

(500, 51, 32)


"Rolling the dice" so that we have a tensor where each matrix is (n PDEs x n QUAD nodes) and the third dimension is n FEM nodes

In [ ]:
#expected_output_rshpd = np.swapaxes(expected_output_rshpd, 0, 1)
#print(expected_output_rshpd.shape) # n FEM nodes x n PDEs x n QUAD nodes

Reshaping input

In [ ]:
input_rshpd = input.reshape([n_PDEs, n_FEMnodes, n_params, data_size])
print(input_rshpd.shape) # n PDEs x n FEM nodes x n params x n QUAD nodes

(500, 51, 5, 32)


In [ ]:
input_rshpd = np.swapaxes(input_rshpd, 2, 3)
print(input_rshpd.shape) # n FEM nodes x n PDEs x n params x n QUAD nodes

(500, 51, 32, 5)


The training data should be interpreted as input and expected output


the input is given by the terms (mu, beta1, beta2, sigma, force) of the ADR problem  computed in the grid (n FEM nodes x n QUAD nodes)

the output is given by the kind-of-solution (...specify how it has been got....) evaluated in the grid (n FEM nodes x n QUAD nodes)

In [ ]:
x_train = input_rshpd
#x_train = np.expand_dims(x_train, 4) 
y_train = expected_output_rshpd

print(x_train.shape)
print(y_train.shape) 

(500, 51, 32, 5)
(500, 51, 32)


training for all n_FEMnodes, the tuning of the width of the window is done within the layers conv1D

In [ ]:
input_shape = (n_FEMnodes, data_size, n_params)
model = tf.keras.Sequential()
layer1 = tf.keras.Input(shape=input_shape)
model.add(layer1)
layer2 = tf.keras.layers.Flatten()
model.add(layer2)
layer3 = tf.keras.layers.Dense(1000, activation="relu", activity_regularizer=tf.keras.regularizers.L2(1e-5))
model.add(layer3)
layer4 = tf.keras.layers.Dense(1000, activation="relu", activity_regularizer=tf.keras.regularizers.L2(1e-5))
model.add(layer4)
layer5 = tf.keras.layers.Dense(1000, activation="relu", activity_regularizer=tf.keras.regularizers.L2(1e-5))
model.add(layer5)
layer6 = tf.keras.layers.Dense(n_FEMnodes * data_size, activity_regularizer=tf.keras.regularizers.L2(1e-5))
model.add(layer6)
layer7 = tf.keras.layers.Reshape((n_FEMnodes, data_size))
model.add(layer7)
model.output_shape

(None, 51, 32)

In [ ]:
model.summary()

In [ ]:
#tf.keras.utils.plot_model(model, "model_shape_info.png", show_shapes=True)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4), 
              #loss = "mse"
              loss = "mae"
              #,metrics=tf.keras.metrics.MeanSquaredLogarithmicError( name='msl_err', dtype=None)
              )

In [ ]:
batch_size = 20
epochs = 20
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split = 0.2)

Epoch 1/20
20/20 [==============================] - 2s 90ms/step - loss: 0.3188 - val_loss: 0.0045
Epoch 2/20
20/20 [==============================] - 2s 82ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 3/20
20/20 [==============================] - 2s 82ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 4/20
20/20 [==============================] - 2s 81ms/step - loss: 0.0017 - val_loss: 0.0019
Epoch 5/20
20/20 [==============================] - 2s 81ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 6/20
20/20 [==============================] - 2s 81ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 7/20
20/20 [==============================] - 2s 82ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 8/20
20/20 [==============================] - 2s 83ms/step - loss: 0.0015 - val_loss: 0.0017
Epoch 9/20
20/20 [==============================] - 2s 82ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 10/20
20/20 [==============================] - 2s 83ms/step - loss: 0.0015 - val_loss: 0.0016
Epoch 11/

Extract testing data the same way

In [ ]:
#testing data

# number of FEM nodes (control nodes in Higa setting)
n_FEMnodes = 51
# number of parameters(mu, beta1, beta2, sigma, f)
n_params = 5
# number of PDEs problems considered (CHANGED WRT TRAINING STAGE OF COURSE)
t_PDEs = 1
# number of quadrature nodes for each FEM node
data_size = 32 #(expected_output.shape[1])

#import expected output dataset
t_expected_output = np.loadtxt('TestOutput.txt')
print(t_expected_output.shape) # n FEM nodes x t PDEs x n QUAD nodes

# import input dataset (think about importing n PDEs files instead SolutionInput_1.txt ... SolutionInput_nPDEs.txt)
t_input = np.loadtxt('TestInput.txt')
print(t_input.shape) # n FEM nodes x n PDEs x t params x n QUAD nodes

(51, 32)
(255, 32)


Reshaping testing expected output

In [ ]:
t_expected_output_rshpd = t_expected_output.reshape([t_PDEs, n_FEMnodes, data_size])
print(t_expected_output_rshpd.shape) # t PDEs x n FEM nodes x n QUAD nodes

(1, 51, 32)


In [ ]:
#t_expected_output_rshpd = np.swapaxes(t_expected_output_rshpd, 0, 1)
#print(t_expected_output_rshpd.shape) # t FEM nodes x n PDEs x n QUAD nodes

Reshaping testing input

In [ ]:
t_input_rshpd = t_input.reshape([t_PDEs, n_FEMnodes, n_params, data_size])
print(t_input_rshpd.shape) # t PDEs x n FEM nodes x n params x n QUAD nodes

(1, 51, 5, 32)


In [ ]:
t_input_rshpd = np.swapaxes(t_input_rshpd, 2, 3)
print(t_input_rshpd.shape) # t FEM nodes x n PDEs x n params x n QUAD nodes

(1, 51, 32, 5)


Evaluate the model through the testing dataset

In [ ]:
x_test = t_input_rshpd
y_test = t_expected_output_rshpd

print(x_test.shape)
print(y_test.shape) 

(1, 51, 32, 5)
(1, 51, 32)


In [ ]:
test_loss = model.evaluate(x_test, y_test)
#print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

1/1 [==============================] - 0s 20ms/step - loss: 0.0250
Test loss: 0.025034448131918907


Predict the output for the desidered input

In [ ]:
result = model.predict(x_test, batch_size=50)
print(result.shape)

(1, 51, 32)


In [ ]:
result

Write a file containing the results of the output of the CNN on the input x_test

In [ ]:
result = result.reshape((n_FEMnodes, data_size))
f = open("output_ML.txt", "w")

for i in range(result.shape[0]):
  for j in range(result.shape[1]):
    f.write(str(result[i][j]))
    f.write("  ")
  f.write("\n")

f.close()